In [175]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import random
import os

In [176]:
# Reading data files from my computer

def get_data():
    directory_pwd = '/Users/charbel/Desktop/COVID Hackathon/'
    directory = os.fsencode(directory_pwd)
    data = []
    labels = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            full_path = directory_pwd + filename
            df = pd.read_csv(full_path)
            df = df[df.Province_State != "Recovered"]
            df = df.drop(["Lat", "Long_", "Last_Update", "Province_State", "Country_Region", "UID", "ISO3", "FIPS", "Deaths", "Active"], 1)
            df = df.fillna(df.median())
            labels.append(df['Confirmed'].to_numpy())
            df = df.drop(['Confirmed'], 1)
            data.append(df.to_numpy())
    return data, labels

data, labels = get_data()
data = np.stack(data)
print("data", data.shape)
labels = np.stack(labels)
print("labels", labels.shape)
# (# of days, #of states/province, #of features)
        

data (104, 58, 7)
labels (104, 58)


In [215]:
class RNN(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.rnn = nn.GRU(input_size=58*7,
                          hidden_size=hidden_size,
                          batch_first=True)
        self.proj = nn.Linear(in_features=hidden_size,
                              out_features=58)
    
    def forward(self, inp):
        X = inp.reshape([-1,58*7]).unsqueeze(dim=0)
        out, last_hidden = self.rnn(X)
        out_seq = self.proj(out)
        return out_seq.squeeze()

In [216]:
rnn = RNN(406)
y = rnn(torch.Tensor(data))
y.shape

torch.Size([104, 58])

In [179]:
y

tensor([[-0.4252,  0.5066,  0.2734,  ...,  0.4183, -0.4099,  0.1718],
        [-0.4252,  0.5066,  0.2734,  ...,  0.4183, -0.4099,  0.1718],
        [-0.1855,  0.0461,  0.0728,  ...,  0.5212, -0.1422,  0.1933],
        ...,
        [ 0.3490, -0.3816, -0.1877,  ...,  0.5307,  0.3856,  0.5428],
        [-0.1486,  0.4895,  0.1528,  ..., -0.2950,  0.3384,  0.2921],
        [-0.1486,  0.4895,  0.1528,  ..., -0.2950,  0.3384,  0.2921]],
       grad_fn=<SqueezeBackward0>)